# Check_predicted_data

Notebook used to check the predicted data coming from the pre-processing pipeline.


In [ ]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.theodolite_function as theodo_f

# Important: set path to deployment data
path = "../data/20220717-5/"

# 1. Static GCP calibration result and accuracy


In [ ]:
# Function used to show the resection accuracy
# Read markers file of grand axe and show them

%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)

path_file_GCP = "total_stations/GCP.txt"
file_name = path+path_file_GCP

trimble_1, trimble_2, trimble_3, T_1_grand, T_2_grand, T_3_grand = theodo_u.read_marker_file(file_name, 1, 1)

tp1 = T_1_grand@trimble_1
tp2 = T_2_grand@trimble_2
tp3 = T_3_grand@trimble_3

save_tf = False
name_file = path + "list_tf/GCP_result_all_points_considered.csv"
if(save_tf):
    theodo_u.save_tf(T_1_grand, T_2_grand, T_3_grand, name_file)

error = []
for i,j,k in zip(tp1,tp2,tp3):
    dist_12 = np.linalg.norm(i-j)
    dist_13 = np.linalg.norm(i-k)
    dist_23 = np.linalg.norm(k-j)
    error.append(dist_12)
    error.append(dist_13)
    error.append(dist_23)
print("Average error during the calibration: ", round(np.mean(error)*1000,3), "mm")

plot_3d = 0   # Variable to plot in 3d, 0 = 2D, 1 = 3D
save_pdf = 0   # Variable to save file in pdf
file_pdf = path + "figures/GCP_calibration_result.pdf"   # file name to save plot in pdf
theodo_p.plot_trajectories_prism(3, tp1, tp2, tp3, T_1_grand, T_2_grand, T_3_grand, plot_3d, save_pdf,file_pdf,1)

# 2. Predicted data

In [ ]:
# Read predicted prism trajectories
theodo_u = importlib.reload(theodo_u)

path_type = "filtered_prediction/"
file_name = "f-2-1-1-1-6-0-L_"

trimble_1 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"1.csv"))
trimble_2 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"2.csv"))
trimble_3 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"3.csv"))
print("Number of points: ", len(trimble_1),len(trimble_2),len(trimble_3))

In [ ]:
# Apply rigid transformations coming from GCP results
theodo_u = importlib.reload(theodo_u)
tp1 = (T_1_grand@(trimble_1.T)[1:5]).T
tp2 = (T_2_grand@(trimble_2.T)[1:5]).T
tp3 = (T_3_grand@(trimble_3.T)[1:5]).T

In [ ]:
# Display prism trajectories
%matplotlib notebook
plt.figure(figsize=(10,3))
plt.scatter(tp1[:,0],tp1[:,1], color='b')
plt.scatter(tp2[:,0],tp2[:,1], color='r')
plt.scatter(tp3[:,0],tp3[:,1], color='g')
plt.show()